![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.svg)
# Pipeline 16 Notebook - AutoAI Notebook v1.22.6

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimal results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a>. 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for retrieving data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.10 and scikit-learn 1.1.1.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data](#read)<br>
&nbsp;&nbsp;[Train and test data split](#split)<br>
&nbsp;&nbsp;[Create pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
**[Store the model](#saving)**<br>
**[Summary and next steps](#summary_and_next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm-watsonx-ai,
 - autoai-libs,
 - scikit-learn,
 - lightgbm


In [ ]:
!pip install ibm-watsonx-ai | tail -n 1
!pip install autoai-libs | tail -n 1
!pip install scikit-learn==1.1.1 | tail -n 1
!pip install -U 'lale>=0.7,<0.8' | tail -n 1
!pip install lightgbm==3.3.2 | tail -n 1
!pip install pandas

Filter warnings for this notebook.

In [ ]:
import warnings

warnings.filterwarnings('ignore')

## Set `n_jobs` parameter to the number of available CPUs

In [ ]:
import os, ast
CPU_NUMBER = 4
if 'RUNTIME_HARDWARE_SPEC' in os.environ:
    CPU_NUMBER = int(ast.literal_eval(os.environ['RUNTIME_HARDWARE_SPEC'])['num_cpu'])

<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

**Note**: If reading data results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`. 

It may be necessary to use methods for initial data pre-processing like: e.g. `DataFrame.dropna()`, `DataFrame.drop_duplicates()`, `DataFrame.sample()`, and outliers handling:

```
from autoai_libs.utils.outliers_mitigation import remove_outliers

df = remove_outliers(df, columns=[experiment_metadata['prediction_column']])
```


In [ ]:
import pandas
dataset = pandas.read_csv("../datasets/Life_Expectancy_Data.csv")
dataset.drop_duplicates()
dataset.sample()

In [ ]:
from autoai_libs.utils.outliers_mitigation import remove_outliers

df = remove_outliers(dataset, columns=["Life expectancy "])
df = df.dropna(subset=['Life expectancy '])
X = df.drop('Life expectancy ', axis=1)
y = df['Life expectancy ']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=33)

<a id="preview_model_to_python_code"></a>
## Create pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [ ]:
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TA1
import autoai_libs.utils.fc_methods
from autoai_libs.cognito.transforms.transform_utils import FS1
from autoai_libs.cognito.transforms.transform_utils import TGen
import autoai_libs.cognito.transforms.transform_extras
from lightgbm.sklearn import LGBMRegressor

#### Pre-processing & Estimator.

In [ ]:
numpy_column_selector_0 = NumpyColumnSelector(columns=[0, 1, 2, 7, 11, 13])
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "char_str", "float_int_num", "char_str", "float_int_num",
        "float_int_num", "float_int_num",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[
        [], [], [],
        [float("nan")],
        [float("nan")],
        [float("nan")],
    ],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    filling_values=float("nan"), missing_values=[float("nan")]
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"), 100001, float("nan"), 100001, 100001, 100001,
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    missing_values=float("nan"),
    sklearn_version_family="1",
    strategy="most_frequent",
)
cat_encoder = CatEncoder(
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="1",
    encoding="ordinal",
    categories="auto",
)
pipeline_0 = make_pipeline(
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
numpy_column_selector_1 = NumpyColumnSelector(
    columns=[3, 4, 5, 6, 8, 9, 10, 12, 14, 15, 16, 17, 18, 19, 20]
)
float_str2_float = FloatStr2Float(
    dtypes_list=[
        "float_int_num", "float_int_num", "float_num", "float_num",
        "float_int_num", "float_num", "float_int_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num",
    ],
    missing_values_reference_list=[float("nan")],
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    filling_values=float("nan"), missing_values=[float("nan")]
)
num_imputer = NumImputer(missing_values=float("nan"), strategy="median")
opt_standard_scaler = OptStandardScaler(use_scaler_flag=False)
pipeline_1 = make_pipeline(
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        0, 1, 2, 7, 11, 13, 3, 4, 5, 6, 8, 9, 10, 12, 14, 15, 16, 17, 18, 19,
        20,
    ],
)
ta1 = TA1(
    fun=np.sqrt,
    name="sqrt",
    datatypes=["numeric"],
    feat_constraints=[
        autoai_libs.utils.fc_methods.is_non_negative,
        autoai_libs.utils.fc_methods.is_not_categorical,
    ],
    col_names=[
        "Country", "Year", "Status", "Adult Mortality", "infant deaths",
        "Alcohol", "percentage expenditure", "Hepatitis B", "Measles ",
        " BMI ", "under-five deaths ", "Polio", "Total expenditure",
        "Diphtheria ", " HIV/AIDS", "GDP", "Population",
        " thinness  1-19 years", " thinness 5-9 years",
        "Income composition of resources", "Schooling",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1_0 = FS1(
    cols_ids_must_keep=range(0, 21),
    additional_col_count_to_keep=20,
    ptype="regression",
)
t_gen = TGen(
    fun=autoai_libs.cognito.transforms.transform_extras.NXOR,
    name="nxor",
    arg_count=2,
    datatypes_list=[["numeric"], ["numeric"]],
    feat_constraints_list=[
        [autoai_libs.utils.fc_methods.is_not_categorical],
        [autoai_libs.utils.fc_methods.is_not_categorical],
    ],
    col_names=[
        "Country", "Year", "Status", "Adult Mortality", "infant deaths",
        "Alcohol", "percentage expenditure", "Hepatitis B", "Measles ",
        " BMI ", "under-five deaths ", "Polio", "Total expenditure",
        "Diphtheria ", " HIV/AIDS", "GDP", "Population",
        " thinness  1-19 years", " thinness 5-9 years",
        "Income composition of resources", "Schooling", "sqrt(Country)",
        "sqrt(Adult Mortality)", "sqrt(infant deaths)", "sqrt(Alcohol)",
        "sqrt(percentage expenditure)", "sqrt(Hepatitis B)", "sqrt(Measles )",
        "sqrt( BMI )", "sqrt(under-five deaths )", "sqrt(Polio)",
        "sqrt(Total expenditure)", "sqrt(Diphtheria )", "sqrt( HIV/AIDS)",
        "sqrt(GDP)", "sqrt(Population)", "sqrt( thinness  1-19 years)",
        "sqrt( thinness 5-9 years)", "sqrt(Income composition of resources)",
        "sqrt(Schooling)",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"),
    ],
)
fs1_1 = FS1(
    cols_ids_must_keep=range(0, 21),
    additional_col_count_to_keep=20,
    ptype="regression",
)
lgbm_regressor = LGBMRegressor(
    boosting_type="dart",
    class_weight="balanced",
    colsample_bytree=0.3872734260973607,
    learning_rate=0.5275297408619404,
    min_child_samples=30,
    min_child_weight=0.008066432867620208,
    n_estimators=823,
    n_jobs=CPU_NUMBER,
    num_leaves=128,
    random_state=33,
    reg_alpha=0.9719659600396976,
    reg_lambda=0.8935398429345613,
    subsample=0.09146847544780322,
)


#### Pipeline.

In [ ]:
pipeline = make_pipeline(
    union, numpy_permute_array, ta1, fs1_0, t_gen, fs1_1, lgbm_regressor
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [ ]:
from sklearn.metrics import make_scorer

from autoai_libs.scorers.scorers import neg_root_mean_squared_error

scorer = make_scorer(neg_root_mean_squared_error)

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [ ]:
pipeline.fit(X_train.values, y_train.values.ravel());

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [ ]:
score = scorer(pipeline, X_test.values, y_test.values)
print(score)

In [ ]:
pipeline.predict(X_test.values[:5])

<a id="summary_and_next_steps"></a>
# Summary and next steps
You successfully completed this notebook!
You learned how to use AutoAI pipeline definition to train the model.
Check out our [Online Documentation](https://www.ibm.com/cloud/watson-studio/autoai) for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>

___